In [1]:
import numpy as np
import time
import cv2
import os 
import matplotlib.pyplot as plt
import scipy
from scipy import signal as sp
from mss import mss
from PIL import Image

# makes plot pop out
%matplotlib tk

In [2]:
def compute_fft(s, sampling_rate, n = None, scale_amplitudes = True):
    '''Computes an FFT on signal s using numpy.fft.fft.
    
       Parameters:
        s (np.array): the signal
        sampling_rate (num): sampling rate
        n (integer): If n is smaller than the length of the input, the input is cropped. If n is 
            larger, the input is padded with zeros. If n is not given, the length of the input signal 
            is used (i.e., len(s))
        scale_amplitudes (boolean): If true, the spectrum amplitudes are scaled by 2/len(s)
    '''
    if n == None:
        n = len(s)
        
    fft_result = np.fft.fft(s, n)
    num_freq_bins = len(fft_result)
    fft_freqs = np.fft.fftfreq(num_freq_bins, d = 1 / sampling_rate)
    half_freq_bins = num_freq_bins // 2
 
    fft_freqs = fft_freqs[:half_freq_bins]
    fft_result = fft_result[:half_freq_bins]
    fft_amplitudes = np.abs(fft_result)
    
    if scale_amplitudes is True:
        fft_amplitudes = 2 * fft_amplitudes / (len(s))
    
    return (fft_freqs, fft_amplitudes)

In [3]:
# version 0.2
#mp_face_mesh = mp.solutions.face_mesh
#face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False) #meant for video feed

def select_point(event, x ,y, flags, param):
    global point, point_selected, old_points;
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x,y)
        point_selected=True
        old_points = np.array([[x,y,]], dtype=np.float32)
        

sct = mss()
bounding_box = {'top': 355, 'left': 100, 'width': 200, 'height': 200}
sct_img = sct.grab(bounding_box)
color_img = np.array(sct_img)
old_gray = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)

        
# lucas kanade params
lk_param = dict(winSize=(100,100), maxLevel=64, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))


cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)
point_selected = False;
point = ()   
old_points = np.array([[]])
rrSignal = []

# hook the function to be called in case of mouse click
cv2.setMouseCallback("Frame", select_point)

while True:
    sumofPixels=0
    sct_img = sct.grab(bounding_box)
    color_img = np.array(sct_img)
    gray_frame = cv2.cvtColor(color_img, cv2.COLOR_BGR2GRAY)
    
    # get the size of the frame
    height, width, channel = color_img.shape
    
    # create a all-zeros mask, we are doing this to get all the pixels covered by the circle
    mask = np.zeros((height, width), np.uint8)
    
    crop=np.zeros((7,7));
    
    if point_selected is True:
        cv2.circle(color_img, point, 4, (0, 5, 255), -1) #BGR Format
        
        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, gray_frame, old_points, None, **lk_param) 
        old_gray = gray_frame.copy()
        old_points = new_points
        x,y = new_points.ravel()
        
        circleRadius = 5
        # draw the moving pint as optics flow
        cv2.circle(color_img, (int(x),int(y)),circleRadius,(255,0,0),-1)
        
        # create the circle with new point over the mask as well
        circle_mask = cv2.circle(mask, (int(x), int(y)),circleRadius,(255,0,0),-1)
        masked_data = cv2.bitwise_and(gray_frame,gray_frame,mask=circle_mask)
        
        # code to get all pixel location inside the ROI in the optical flow
        tom, thres = cv2.threshold(mask, 1,255, cv2.THRESH_BINARY)
        contours = cv2.findContours(thres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        x,y,w,h = cv2.boundingRect(contours[0][0])
        crop = masked_data[y:y+h,x:x+w]
        
        
    sumofPixels = np.sum(crop[:,:])
    rrSignal.append(sumofPixels)
    
    if(len(rrSignal)>=120):
        size = 15
        window = sp.gaussian(M=size, std=6)
        window /= window.sum()
        filteredA = np.convolve(np.array(rrSignal[-120:]), window, mode='same')

        F,A = compute_fft(filteredA, 25, n = None, scale_amplitudes = True)
        highestFreq = F[np.argmax(A[1:])+1]

        #print("Breathing Rate = {:.2f}".format(highestFreq*60))  

        plt.plot(rrSignal[-120:], 'b-')
        plt.title("Respiration Rate",fontdict={'fontsize': 20})
        plt.xlabel('Frame Number')
        plt.show()
        plt.pause(0.001)

        plt.clf()

    # Create the haar cascade
    #noseCascade = cv2.CascadeClassifier("haarcascade_mcs_nose.xml")
    #nose = noseCascade.detectMultiScale(gray)
                        
    #for (ex,ey,ew,eh) in nose:
    #    cv2.rectangle(color_img,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    
    cv2.imshow('Frame', color_img)
    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        cv2.destroyAllWindows()
        break

IndexError: tuple index out of range